In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime
import uuid
from typing import List, Dict

# =============================================
# ABSTRACTION: Abstract Base Classes
# =============================================

class Person(ABC):
    """Abstract base class for all persons in the school system"""

    def __init__(self, name: str, email: str, phone: str):
        self._id = str(uuid.uuid4())[:8]  # ENCAPSULATION: Protected attribute
        self._name = name
        self._email = email
        self._phone = phone
        self._registration_date = datetime.now()

    # ENCAPSULATION: Using properties to control access to attributes
    @property
    def id(self):
        return self._id

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        if value and isinstance(value, str):
            self._name = value

    @property
    def email(self):
        return self._email

    @property
    def registration_date(self):
        return self._registration_date

    # ABSTRACTION: Abstract methods to be implemented by subclasses
    @abstractmethod
    def get_role(self):
        """Return the role of the person in the school"""
        pass

    @abstractmethod
    def display_info(self):
        """Display complete information about the person"""
        pass


class Course(ABC):
    """Abstract base class for all courses in the school"""

    def __init__(self, course_code: str, course_name: str, credits: int):
        self._course_code = course_code
        self._course_name = course_name
        self._credits = credits
        self._max_students = 30  # ENCAPSULATION: Default capacity

    @property
    def course_code(self):
        return self._course_code

    @property
    def course_name(self):
        return self._course_name

    @property
    def credits(self):
        return self._credits

    @property
    def max_students(self):
        return self._max_students

    # ABSTRACTION: Abstract method for course-specific details
    @abstractmethod
    def get_course_type(self):
        """Return the type of course"""
        pass

    @abstractmethod
    def get_prerequisites(self):
        """Return course prerequisites"""
        pass


# =============================================
# INHERITANCE: Person Subclasses
# =============================================

class Student(Person):
    """Student class inheriting from Person - represents a student in the school"""

    def __init__(self, name: str, email: str, phone: str, grade_level: str):
        super().__init__(name, email, phone)
        self._grade_level = grade_level
        self._enrolled_courses = []  # ENCAPSULATION: Private list
        self._gpa = 0.0
        self._total_credits = 0

    # POLYMORPHISM: Implementing abstract methods specifically for Student
    def get_role(self):
        return "Student"

    def display_info(self):
        info = f"Student ID: {self._id}\nName: {self._name}\nGrade: {self._grade_level}\n"
        info += f"Email: {self._email}\nGPA: {self._gpa:.2f}\nCredits: {self._total_credits}"
        return info

    # Student-specific methods
    def enroll_in_course(self, course) -> bool:
        """Enroll student in a course if there's space"""
        if len(self._enrolled_courses) < 5:  # Max 5 courses per student
            if course not in self._enrolled_courses:
                self._enrolled_courses.append(course)
                print(f"Successfully enrolled {self._name} in {course.course_name}")
                return True
        else:
            print(f"Cannot enroll {self._name} in more than 5 courses")
        return False

    def drop_course(self, course) -> bool:
        """Drop a course from student's schedule"""
        if course in self._enrolled_courses:
            self._enrolled_courses.remove(course)
            print(f"Successfully dropped {course.course_name} for {self._name}")
            return True
        return False

    def calculate_gpa(self, grades: Dict[str, float]) -> float:
        """Calculate GPA based on course grades"""
        if not grades:
            return 0.0

        total_points = 0
        total_credits = 0

        for course_code, grade in grades.items():
            # Find course by code and get its credits
            course = next((c for c in self._enrolled_courses if c.course_code == course_code), None)
            if course:
                total_points += grade * course.credits
                total_credits += course.credits

        if total_credits > 0:
            self._gpa = total_points / total_credits
            self._total_credits = total_credits

        return self._gpa

    def get_enrolled_courses(self):
        """Get list of enrolled courses (ENCAPSULATION: returning copy)"""
        return self._enrolled_courses.copy()


class Teacher(Person):
    """Teacher class inheriting from Person - represents a teacher in the school"""

    def __init__(self, name: str, email: str, phone: str, department: str):
        super().__init__(name, email, phone)
        self._department = department
        self._assigned_courses = []  # ENCAPSULATION: Private list

    # POLYMORPHISM: Implementing abstract methods specifically for Teacher
    def get_role(self):
        return "Teacher"

    def display_info(self):
        info = f"Teacher ID: {self._id}\nName: {self._name}\nDepartment: {self._department}\n"
        info += f"Email: {self._email}\nCourses Assigned: {len(self._assigned_courses)}"
        return info

    # Teacher-specific methods
    def assign_course(self, course) -> bool:
        """Assign a course to the teacher"""
        if course not in self._assigned_courses:
            self._assigned_courses.append(course)
            print(f"Assigned {course.course_name} to {self._name}")
            return True
        return False

    def get_assigned_courses(self):
        """Get list of assigned courses"""
        return self._assigned_courses.copy()


# =============================================
# INHERITANCE: Course Subclasses
# =============================================

class CoreCourse(Course):
    """CoreCourse class inheriting from Course - represents required courses"""

    def __init__(self, course_code: str, course_name: str, credits: int, department: str):
        super().__init__(course_code, course_name, credits)
        self._department = department
        self._is_required = True

    # POLYMORPHISM: Implementing abstract methods specifically for CoreCourse
    def get_course_type(self):
        return "Core Course"

    def get_prerequisites(self):
        return "No prerequisites - Required course"

    def display_course_info(self):
        info = f"{self._course_code}: {self._course_name}\n"
        info += f"Type: {self.get_course_type()}\nCredits: {self._credits}\n"
        info += f"Department: {self._department}\nRequired: {self._is_required}"
        return info


class ElectiveCourse(Course):
    """ElectiveCourse class inheriting from Course - represents optional courses"""

    def __init__(self, course_code: str, course_name: str, credits: int, department: str, prerequisites: List[str]):
        super().__init__(course_code, course_name, credits)
        self._department = department
        self._prerequisites = prerequisites
        self._is_required = False

    # POLYMORPHISM: Implementing abstract methods specifically for ElectiveCourse
    def get_course_type(self):
        return "Elective Course"

    def get_prerequisites(self):
        if self._prerequisites:
            return f"Prerequisites: {', '.join(self._prerequisites)}"
        return "No prerequisites"

    def display_course_info(self):
        info = f"{self._course_code}: {self._course_name}\n"
        info += f"Type: {self.get_course_type()}\nCredits: {self._credits}\n"
        info += f"Department: {self._department}\nRequired: {self._is_required}\n"
        info += f"{self.get_prerequisites()}"
        return info


# =============================================
# COMPOSITION: School Management System
# =============================================

class SchoolRegistrationPortal:
    """Main school registration system that composes all other classes"""

    def __init__(self, school_name: str):
        self._school_name = school_name
        self._students = {}  # ENCAPSULATION: Private dictionaries
        self._teachers = {}
        self._courses = {}
        self._academic_year = "2024-2025"

    # Student management methods
    def register_student(self, name: str, email: str, phone: str, grade_level: str) -> str:
        """Register a new student and return student ID"""
        student = Student(name, email, phone, grade_level)
        self._students[student.id] = student
        print(f"Successfully registered student: {name} (ID: {student.id})")
        return student.id

    def enroll_student_in_course(self, student_id: str, course_code: str) -> bool:
        """Enroll a student in a specific course"""
        student = self._students.get(student_id)
        course = self._courses.get(course_code)

        if student and course:
            return student.enroll_in_course(course)
        else:
            print("Student or course not found")
            return False

    # Teacher management methods
    def hire_teacher(self, name: str, email: str, phone: str, department: str) -> str:
        """Hire a new teacher and return teacher ID"""
        teacher = Teacher(name, email, phone, department)
        self._teachers[teacher.id] = teacher
        print(f"Successfully hired teacher: {name} (ID: {teacher.id})")
        return teacher.id

    def assign_teacher_to_course(self, teacher_id: str, course_code: str) -> bool:
        """Assign a teacher to a specific course"""
        teacher = self._teachers.get(teacher_id)
        course = self._courses.get(course_code)

        if teacher and course:
            return teacher.assign_course(course)
        else:
            print("Teacher or course not found")
            return False

    # Course management methods
    def add_course(self, course_type: str, **kwargs) -> str:
        """Add a new course to the system"""
        course = None

        # POLYMORPHISM: Creating different course types through common interface
        if course_type.lower() == "core":
            course = CoreCourse(
                kwargs['course_code'],
                kwargs['course_name'],
                kwargs['credits'],
                kwargs['department']
            )
        elif course_type.lower() == "elective":
            course = ElectiveCourse(
                kwargs['course_code'],
                kwargs['course_name'],
                kwargs['credits'],
                kwargs['department'],
                kwargs.get('prerequisites', [])
            )
        else:
            raise ValueError("Invalid course type")

        self._courses[course.course_code] = course
        print(f"Successfully added course: {course.course_name} ({course.course_code})")
        return course.course_code

    # Reporting methods
    def generate_student_report(self, student_id: str):
        """Generate a comprehensive report for a student"""
        student = self._students.get(student_id)
        if not student:
            print("Student not found")
            return

        print(f"\n{'='*50}")
        print(f"STUDENT REPORT - {self._school_name}")
        print(f"{'='*50}")
        print(student.display_info())
        print(f"\nEnrolled Courses ({len(student.get_enrolled_courses())}):")
        for course in student.get_enrolled_courses():
            print(f"  - {course.course_code}: {course.course_name} ({course.credits} credits)")
        print(f"{'='*50}")

    def generate_teacher_report(self, teacher_id: str):
        """Generate a report for a teacher"""
        teacher = self._teachers.get(teacher_id)
        if not teacher:
            print("Teacher not found")
            return

        print(f"\n{'='*50}")
        print(f"TEACHER REPORT - {self._school_name}")
        print(f"{'='*50}")
        print(teacher.display_info())
        print(f"\nAssigned Courses ({len(teacher.get_assigned_courses())}):")
        for course in teacher.get_assigned_courses():
            print(f"  - {course.course_code}: {course.course_name}")
        print(f"{'='*50}")

    def display_all_courses(self):
        """Display all available courses"""
        print(f"\n{'='*60}")
        print(f"AVAILABLE COURSES - {self._school_name} ({self._academic_year})")
        print(f"{'='*60}")
        for course_code, course in self._courses.items():
            print(course.display_course_info())
            print("-" * 40)


# =============================================
# DEMONSTRATION: Showing OOP Features in Action
# =============================================

def main():
    """Main function to demonstrate the School Registration Portal"""

    # Create school registration portal
    school = SchoolRegistrationPortal("Python High School")

    print("=== SCHOOL REGISTRATION PORTAL DEMONSTRATION ===")
    print("Showing all four OOP pillars: Encapsulation, Inheritance, Polymorphism, and Abstraction\n")

    # ==================================================================
    # DEMONSTRATION 1: COURSE CREATION (POLYMORPHISM & INHERITANCE)
    # ==================================================================
    print("1. CREATING COURSES (POLYMORPHISM & INHERITANCE):")
    print("-" * 40)

    # Create core courses
    school.add_course("core",
                     course_code="MATH101",
                     course_name="Algebra I",
                     credits=4,
                     department="Mathematics")

    school.add_course("core",
                     course_code="ENG101",
                     course_name="English Literature",
                     credits=3,
                     department="English")

    # Create elective courses with prerequisites
    school.add_course("elective",
                     course_code="CS201",
                     course_name="Advanced Programming",
                     credits=4,
                     department="Computer Science",
                     prerequisites=["CS101", "MATH101"])

    school.add_course("elective",
                     course_code="ART101",
                     course_name="Introduction to Art",
                     credits=2,
                     department="Arts",
                     prerequisites=[])

    print("\n" + "="*60 + "\n")

    # ==================================================================
    # DEMONSTRATION 2: STUDENT REGISTRATION (ENCAPSULATION)
    # ==================================================================
    print("2. STUDENT REGISTRATION (ENCAPSULATION):")
    print("-" * 40)

    # Register students
    alice_id = school.register_student("Alice Johnson", "alice@email.com", "555-0101", "10th Grade")
    bob_id = school.register_student("Bob Smith", "bob@email.com", "555-0102", "11th Grade")

    print("\n" + "="*60 + "\n")

    # ==================================================================
    # DEMONSTRATION 3: TEACHER HIRING (INHERITANCE)
    # ==================================================================
    print("3. TEACHER HIRING (INHERITANCE):")
    print("-" * 40)

    # Hire teachers
    mr_wilson_id = school.hire_teacher("Mr. Wilson", "wilson@school.edu", "555-0201", "Mathematics")
    ms_davis_id = school.hire_teacher("Ms. Davis", "davis@school.edu", "555-0202", "Computer Science")

    print("\n" + "="*60 + "\n")

    # ==================================================================
    # DEMONSTRATION 4: COURSE ASSIGNMENTS (POLYMORPHISM)
    # ==================================================================
    print("4. COURSE ASSIGNMENTS (POLYMORPHISM):")
    print("-" * 40)

    # Assign teachers to courses
    school.assign_teacher_to_course(mr_wilson_id, "MATH101")
    school.assign_teacher_to_course(ms_davis_id, "CS201")

    # Enroll students in courses
    school.enroll_student_in_course(alice_id, "MATH101")
    school.enroll_student_in_course(alice_id, "ENG101")
    school.enroll_student_in_course(alice_id, "ART101")

    school.enroll_student_in_course(bob_id, "MATH101")
    school.enroll_student_in_course(bob_id, "CS201")

    print("\n" + "="*60 + "\n")

    # ==================================================================
    # DEMONSTRATION 5: GPA CALCULATION & REPORTING (ABSTRACTION)
    # ==================================================================
    print("5. GPA CALCULATION & REPORTING (ABSTRACTION):")
    print("-" * 40)

    # Get student objects for direct operations
    alice = school._students[alice_id]  # Accessing protected member for demo
    bob = school._students[bob_id]

    # Calculate GPA for students (simulated grades)
    alice_grades = {"MATH101": 3.8, "ENG101": 3.5, "ART101": 4.0}
    bob_grades = {"MATH101": 3.2, "CS201": 3.7}

    alice.calculate_gpa(alice_grades)
    bob.calculate_gpa(bob_grades)

    print(f"Alice's GPA: {alice._gpa:.2f}")  # Accessing protected member
    print(f"Bob's GPA: {bob._gpa:.2f}")

    print("\n" + "="*60 + "\n")

    # ==================================================================
    # DEMONSTRATION 6: COMPREHENSIVE REPORTS
    # ==================================================================
    print("6. COMPREHENSIVE REPORTS:")
    print("-" * 40)

    # Generate student reports
    school.generate_student_report(alice_id)
    school.generate_student_report(bob_id)

    # Generate teacher report
    school.generate_teacher_report(ms_davis_id)

    # Display all available courses
    school.display_all_courses()

    # ==================================================================
    # DEMONSTRATION 7: ERROR HANDLING & VALIDATION (ENCAPSULATION)
    # ==================================================================
    print("\n7. ERROR HANDLING & VALIDATION (ENCAPSULATION):")
    print("-" * 40)

    # Try to enroll in non-existent course
    school.enroll_student_in_course(alice_id, "PHYSICS101")

    # Try to assign non-existent teacher
    school.assign_teacher_to_course("INVALID_ID", "MATH101")


if __name__ == "__main__":
    main()

=== SCHOOL REGISTRATION PORTAL DEMONSTRATION ===
Showing all four OOP pillars: Encapsulation, Inheritance, Polymorphism, and Abstraction

1. CREATING COURSES (POLYMORPHISM & INHERITANCE):
----------------------------------------
Successfully added course: Algebra I (MATH101)
Successfully added course: English Literature (ENG101)
Successfully added course: Advanced Programming (CS201)
Successfully added course: Introduction to Art (ART101)


2. STUDENT REGISTRATION (ENCAPSULATION):
----------------------------------------
Successfully registered student: Alice Johnson (ID: e329c4ac)
Successfully registered student: Bob Smith (ID: b47e17c8)


3. TEACHER HIRING (INHERITANCE):
----------------------------------------
Successfully hired teacher: Mr. Wilson (ID: 3494ee7c)
Successfully hired teacher: Ms. Davis (ID: 6b7af68f)


4. COURSE ASSIGNMENTS (POLYMORPHISM):
----------------------------------------
Assigned Algebra I to Mr. Wilson
Assigned Advanced Programming to Ms. Davis
Successfully 